In [43]:
import polars as pl
import numpy as np
import time

In [44]:
df = pl.read_csv("data/customers.csv")
print(df.head())

shape: (5, 12)
┌───────┬────────────┬────────────┬───────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ Index ┆ Customer   ┆ First Name ┆ Last Name ┆ … ┆ Phone 2    ┆ Email     ┆ Subscript ┆ Website   │
│ ---   ┆ Id         ┆ ---        ┆ ---       ┆   ┆ ---        ┆ ---       ┆ ion Date  ┆ ---       │
│ i64   ┆ ---        ┆ str        ┆ str       ┆   ┆ str        ┆ str       ┆ ---       ┆ str       │
│       ┆ str        ┆            ┆           ┆   ┆            ┆           ┆ str       ┆           │
╞═══════╪════════════╪════════════╪═══════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 1     ┆ 4962fdbE6B ┆ Pam        ┆ Sparks    ┆ … ┆ 480-078-05 ┆ nicolas00 ┆ 2020-11-2 ┆ https://n │
│       ┆ fee6D      ┆            ┆           ┆   ┆ 35x889     ┆ @faulkner ┆ 9         ┆ elson.com │
│       ┆            ┆            ┆           ┆   ┆            ┆ -kramer.c ┆           ┆ /         │
│       ┆            ┆            ┆           ┆   ┆            ┆ om        ┆

## 🔴 PHẦN 1: EAGER EXECUTION (CÁCH TIẾP CẬN "NGÂY THƠ")

Trong cách tiếp cận truyền thống này (tương tự Pandas), chúng ta sử dụng `pl.read_csv`.

**Quy trình hoạt động:**
1.  **Đọc toàn bộ:** Máy tính đọc **100% dữ liệu** (hàng triệu dòng) từ ổ cứng và nạp thẳng vào RAM.
2.  **Xử lý tuần tự:** Các lệnh chạy đúng theo thứ tự viết: *Tính toán -> Sắp xếp -> Lọc*.
3.  **Hệ quả:**
    * Nếu ta viết lệnh `sort` (sắp xếp) trước lệnh `filter` (lọc), máy sẽ "ngây thơ" sắp xếp cả triệu dòng dữ liệu, gây tốn CPU và RAM vô ích.
    * Dễ gặp lỗi `MemoryError` nếu dữ liệu lớn hơn RAM.

👉 **Hãy xem code bên dưới chạy chậm như thế nào khi ta cố tình viết thứ tự xử lý chưa tối ưu.**

In [150]:
print("-------------------- BẮT ĐẦU TEST EAGER --------------------")
start = time.time()
# Eager execution
df = pl.read_csv("data/customers.csv")
result = (
    df
    .sort("Last Name")
    .sort("First Name")
    .sort("City")
    .sort("Company")
    .filter(pl.col("Country") == "Macao")
    .filter(pl.col("Subscription Date") >= "2020-01-01")
    .filter(pl.col("Subscription Date") < "2021-01-01")
    .select(["First Name", "Last Name", "Company", "City", "Subscription Date"])
)
print(result)

end = time.time()
print(f"Eager execution time: {end - start} seconds")
print("-------------------- KẾT THÚC TEST EAGER --------------------")

-------------------- BẮT ĐẦU TEST EAGER --------------------
shape: (3_473, 5)
┌────────────┬───────────┬─────────────────────────────┬───────────────────┬───────────────────┐
│ First Name ┆ Last Name ┆ Company                     ┆ City              ┆ Subscription Date │
│ ---        ┆ ---       ┆ ---                         ┆ ---               ┆ ---               │
│ str        ┆ str       ┆ str                         ┆ str               ┆ str               │
╞════════════╪═══════════╪═════════════════════════════╪═══════════════════╪═══════════════════╡
│ Jo         ┆ Bryan     ┆ Abbott and Sons             ┆ Port Chris        ┆ 2020-05-03        │
│ Fernando   ┆ Spence    ┆ Abbott, Farrell and Lester  ┆ Adrianaburgh      ┆ 2020-05-30        │
│ Shaun      ┆ Dennis    ┆ Abbott-Bowen                ┆ East Darrenshire  ┆ 2020-06-01        │
│ Angie      ┆ Medina    ┆ Abbott-Davenport            ┆ Connorbury        ┆ 2020-05-27        │
│ Jonathan   ┆ Garner    ┆ Abbott-Keller        

## 🟢 PHẦN 2: LAZY EXECUTION

Ở đây, chúng ta chuyển sang dùng `pl.scan_csv`. Polars sẽ không chạy ngay mà sẽ lập một Query Plan.

**Query Optimizer:**
Polars sẽ tự động sửa lại code của bạn để chạy nhanh nhất:

1.  **Predicate Pushdown (Đẩy bộ lọc xuống đáy):**
    * Polars tự động dời lệnh `filter` (Lọc) xuống chạy trước lệnh `sort` (Sắp xếp).
    * *Tác dụng:* Loại bỏ rác ngay từ cửa, giảm tải khối lượng công việc cho các bước sau.

2.  **Redundant Sort Removal (Loại bỏ sort thừa):**
    * Nếu bạn lỡ tay viết nhiều lệnh `sort` liên tiếp (ví dụ: sắp xếp theo Tên, rồi lại sắp xếp theo Tuổi...), Polars hiểu rằng các lần sắp xếp trước là vô nghĩa.
    * *Tác dụng:* Nó tự động **XÓA BỎ** tất cả các lệnh `sort` phía trước, chỉ giữ lại lệnh `sort` cuối cùng. (Eager sẽ chạy hết tất cả và tốn gấp n lần thời gian).

3.  **Projection Pushdown:** Chỉ đọc những cột cần thiết, bỏ qua các cột dư thừa trong file.

👉 **Kết quả:** Hãy xem **Query Plan** (`.explain()`) bên dưới. Dù ta viết rất nhiều lệnh `sort`, nhưng trong bảng kế hoạch chỉ còn lại đúng một lệnh duy nhất!

In [156]:
print("-------------------- BẮT ĐẦU TEST LAZY --------------------")
start = time.time()
# Lazy execution

result = (
    pl.scan_csv("data/customers.csv")
    .sort("Last Name")
    .sort("First Name")
    .sort("City")
    .sort("Company")
    .filter(pl.col("Country") == "Macao")
    .filter(pl.col("Subscription Date") >= "2020-01-01")
    .filter(pl.col("Subscription Date") < "2021-01-01")
    .select(["First Name", "Last Name", "Company", "City", "Subscription Date"])
)
print(result.explain())
print(result.collect(streaming=True))
end = time.time()
print(f"Lazy execution time: {end - start} seconds")
print("-------------------- KẾT THÚC TEST LAZY --------------------")

-------------------- BẮT ĐẦU TEST LAZY --------------------
simple π 5/5 ["First Name", "Last Name", ... 3 other columns]
  SORT BY [col("Company")]
    Csv SCAN [data/customers.csv]
    PROJECT 6/12 COLUMNS
    SELECTION: [([([(col("Subscription Date")) >= ("2020-01-01")]) & ([(col("Subscription Date")) < ("2021-01-01")])]) & ([(col("Country")) == ("Macao")])]
    ESTIMATED ROWS: 1738392
shape: (3_473, 5)
┌────────────┬───────────┬─────────────────────────────┬───────────────────┬───────────────────┐
│ First Name ┆ Last Name ┆ Company                     ┆ City              ┆ Subscription Date │
│ ---        ┆ ---       ┆ ---                         ┆ ---               ┆ ---               │
│ str        ┆ str       ┆ str                         ┆ str               ┆ str               │
╞════════════╪═══════════╪═════════════════════════════╪═══════════════════╪═══════════════════╡
│ Jo         ┆ Bryan     ┆ Abbott and Sons             ┆ Port Chris        ┆ 2020-05-03        │
│ Fernan

/var/folders/g_/1t9711s10zz6rrllddr6d1r80000gn/T/ipykernel_95316/2678924851.py:17: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  print(result.collect(streaming=True))


# 🏆 TỔNG KẾT & BÀI HỌC

Qua bài demo so sánh hiệu năng trên tập dữ liệu lớn, chúng ta rút ra 3 điểm cốt lõi về sức mạnh của Polars:

### 1. Tốc độ vượt trội nhờ Laziness
Thay vì làm hùng hục như Eager, chế độ **Lazy** giúp Polars nhìn thấy bức tranh toàn cảnh, từ đó tự động sửa lỗi logic của người lập trình để chọn ra con đường ngắn nhất.
* **Eager:** Chạy theo lệnh (Code sao chạy vậy).
* **Lazy:** Chạy theo ý định (Code sai, Engine tự sửa cho tối ưu).

### 2. Tiết kiệm tài nguyên (RAM & CPU)
Nhờ kỹ thuật **Predicate Pushdown** (Lọc ngay từ gốc), Polars không bao giờ mang "rác" vào bộ nhớ. Điều này cho phép xử lý những file dữ liệu lớn gấp nhiều lần dung lượng RAM của máy (đặc biệt khi kết hợp với chế độ `streaming=True`).

### 3. Lời khuyên thực tế
* Dùng **Eager** (`read_csv`) khi: Debug, khám phá dữ liệu nhỏ, cần xem kết quả ngay từng bước.
* Dùng **Lazy** (`scan_csv`) khi: Xây dựng pipeline xử lý dữ liệu (ETL), làm việc với Big Data, hoặc cần hiệu năng cao nhất.

> *"Polars không chỉ nhanh vì nó được viết bằng Rust, mà nó nhanh vì nó thông minh."*